In [8]:
import numpy as np
import pandas as pd

# Define Sharpe Ratio Function
def sharpe_ratio(returns, risk_free_rate=0.0):
    """ Compute Sharpe Ratio: (Mean Return - Risk-Free Rate) / Std Dev """
    excess_returns = returns - risk_free_rate
    return np.sqrt(252) * excess_returns.mean() / (returns.std() + 1e-8)  # Avoid division by zero

# Simulated Stock Data for Multiple Stocks
np.random.seed(42)
dates = pd.date_range(start="2020-01-01", periods=500, freq="D")
stocks = ["AAPL", "MSFT", "GOOGL", "TSLA", "AMZN", "META", "NVDA", "NFLX", "AMD", "INTC"]

performance_data = []

for stock in stocks:
    # Generate Stock Prices (Simulated)
    prices = 100 + np.cumsum(np.random.randn(500) * 2)  # Random Walk

    df_stock = pd.DataFrame({"Date": dates, "Close": prices})
    df_stock["Daily_Returns"] = df_stock["Close"].pct_change().fillna(0)

    # Buy & Hold Strategy
    df_stock["Buy_Hold_Returns"] = df_stock["Daily_Returns"]

    # Mean Reversion Strategies (Random Signals for Demonstration)
    df_stock["MR_Signal"] = np.random.choice([-1, 0, 1], size=len(df_stock))  # Random -1, 0, 1
    df_stock["MR_Returns"] = df_stock["Daily_Returns"] * df_stock["MR_Signal"].shift(1).fillna(0)

    # Simulated TDQN Portfolio Values (If real values missing)
    df_tdqn = pd.DataFrame({
        "Date": dates,
        "Portfolio_Value_TDQN": 100000 + np.cumsum(np.random.randn(500) * 500)  # Simulated Portfolio
    })
    df_tdqn["Date"] = pd.to_datetime(df_tdqn["Date"])

    # Merge TDQN Portfolio Data
    df_stock = df_stock.merge(df_tdqn[["Date", "Portfolio_Value_TDQN"]], on="Date", how="left")

    # Compute TDQN Returns
    df_stock["TDQN_Returns"] = df_stock["Portfolio_Value_TDQN"].pct_change().fillna(0)

    # Compute Sharpe Ratios
    sharpe_bh = sharpe_ratio(df_stock["Buy_Hold_Returns"])
    sharpe_mr = sharpe_ratio(df_stock["MR_Returns"])
    sharpe_tdqn = sharpe_ratio(df_stock["TDQN_Returns"])

    # Append to Performance Table
    performance_data.append([stock, sharpe_bh, sharpe_mr, sharpe_tdqn])

# Create DataFrame
performance_df = pd.DataFrame(performance_data, columns=["Stock", "Sharpe_B&H", "Sharpe_MR", "Sharpe_TDQN"])

# Print Performance Summary
print(performance_df)


   Stock  Sharpe_B&H  Sharpe_MR  Sharpe_TDQN
0   AAPL    0.249301  -1.865332     0.280558
1   MSFT   -0.121058   0.169753     0.098154
2  GOOGL    0.316671  -0.358382     0.066561
3   TSLA    0.517792   1.076499    -1.058364
4   AMZN    0.504188   0.037187     0.210095
5   META    0.066087  -0.091579     0.242969
6   NVDA   -0.305806  -1.265358     1.195029
7   NFLX   -0.113669  -0.562340    -0.325001
8    AMD   -0.229560   0.009209     0.174593
9   INTC    0.120770   1.310908     0.220255
